# Bot Testing Grounds
This notebook tests the performance of a series of bots from draftsimtools.

### Package Importing

First, we load relevant packages, including the custom draftsimtools module. 

In [10]:
# Imports packages 

import warnings
warnings.filterwarnings('ignore')

import pickle
import ast
import numpy as np
import pandas as pd
from operator import itemgetter
from copy import deepcopy
import json
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data.dataset import Dataset
import time

# Workaround for variable Jupyter directories
import sys
sys.path.append('bots')

import draftsimtools as ds
from draftsimtools import DraftNet

### Data Loading

Next, we set filepaths for raw drafts, the MTG Json file containing detailed info on every card, and for curated draftsim ratings of each card in the current set. In this notebook, we will be only be working with M19 drafts.

To get access to the raw drafts and draftsim rankings, please contact [Dan Troha](https://draftsim.com/contact/).

In [8]:
# Sets pytorch device
device = torch.device("cpu") 

# Sets file paths
jsonPath = "../../data/AllSets.json"
ratingPath = "../../data/standardized_m19/standardized_m19_rating.tsv"
draftPath = "../../data/standardized_m19/drafts_test.pkl"

# Sets file paths for Bayesian bot
pCollPath = "bots_data/bayes_pCoDraft.csv"
pPackPath = "bots_data/bayes_pChoice.csv"
pFullPath = "bots_data/bayes_pFull.csv"
namesPath = "bots_data/bayes_names.csv"

We read in raw drafts and Draftsim card rankings here. We also create a label encoder object to map packs to binary presence/absence vectors, which is necessary for some bots.

In [9]:
# Loads drafts
drafts = None
with open(draftPath, "rb") as f:
    drafts = pickle.load(f)
print(f'Full test dataset length: {len(drafts)}')
    
# Loads ratings
m19_set = pd.read_csv(ratingPath, delimiter="\t", converters={6:ast.literal_eval})

# Label-encodes card names
le = ds.create_le(m19_set["Name"].values)

Full test dataset length: 21590


For demonstration purposes, we subset the full set of testing drafts (~22k) to just 100 drafts. 

In [40]:
# Subsets drafts for faster runtimes - for real testing, use all drafts
subset_drafts = drafts[:1000] 

### Testing Bots

We need to instantiate all of the different drafting agents.

**RandomBot**: Picks cards randomly. 

**RaredraftBot**: Picks the rarest cards in its most-dominant color.

**ClassicBot**: Picks cards with the highest draftsim score in its most-dominant colors. 

**BayesBot**: TODO

**NNetBot**: TODO

**RyanBot**: TODO

In [41]:
# Instantiates heuristic-based bots
bot1 = ds.RandomBot() 
bot2 = ds.RaredraftBot(m19_set) 
bot3 = ds.ClassicBot(m19_set) 
bot4 = ds.BayesBot(le, pCollPath, pPackPath, pFullPath, namesPath)

# Loads neural net from saved pytorch file
test_net = torch.load("bots_data/draftnet_jan19_2020_ep23.pt")
test_net.eval()

# Instantiates neural-network bot
bot5 = ds.NeuralNetBot(test_net, le)

                           Name
0            Abnormal_Endurance
1                Act_of_Treason
2          Aegis_of_the_Heavens
3               Aerial_Engineer
4                 Aether_Tunnel
5        Aethershield_Artificer
6            Ajani's_Last_Stand
7             Ajani's_Pridemate
8               Ajani's_Welcome
9    Ajani_Adversary_of_Tyrants
10                  Alpine_Moon
11        Amulet_of_Safekeeping
12            Angel_of_the_Dawn
13                   Anticipate
14                Apex_of_Power
15       Arcades_the_Strategist
16          Arcane_Encyclopedia
17               Aven_Wind_Mage
18             Aviation_Pioneer
19                     Banefire
20             Blanchwood_Armor
21             Blood_Divination
22                Boggart_Brute
23                   Bogstomper
24                  Bone_Dragon
25                  Bone_to_Ash
26              Brawl-Bash_Ogre
27               Bristling_Boar
28                       Cancel
29           Catalyst_Elemental
..      

Finally, we test all of the different bots against each other by measuring their top-one accuracy on predicting human choices in the subset 100 drafts. The overall accuracy for all bots across all drafts is output, as well as csv files containing bot predictions across all drafts. 

In [ ]:
# Tests all bots in the testing framework
tic = time.time()
tester = ds.BotTester(subset_drafts)
tester.evaluate_bots([bot1, bot2, bot3, bot4, bot5], ["RandomBot", "RaredraftBot", "ClassicBot", "BayesBot", "NNetBot"])
tester.report_evaluations()
tester.write_evaluations()
print(f'Finished testing {len(subset_drafts)} drafts in {round((time.time()-tic)/60):d} minutes.')

### Rank Example Pack

To illustrate how the bots' interface works, we show how the NNetBot ranks cards in a single pack. 

In [7]:
# Instantiates bot tester
tester = ds.BotTester(subset_drafts)

# Create demo collection
demo_collection = tester.drafts[0][0]
demo_pack = tester.drafts[0][1]
demo_x = ds.collection_pack_to_x(demo_collection, demo_pack, le)

# Return the result
result = test_net(demo_x)

# Maps numeric classes to card names and displays result
pack_dict = {str(le.inverse_transform([i])[0]) : float(v.detach().numpy()) for i, v in enumerate(result[0,:]) if v > 0}
display(pack_dict)

{'Aethershield_Artificer': 14.505582809448242,
 'Gallant_Cavalry': 16.059925079345703,
 'Goblin_Instigator': 16.97736167907715,
 'Greenwood_Sentinel': 13.722554206848145,
 'Infectious_Horror': 12.23381519317627,
 'Loxodon_Line_Breaker': 12.887977600097656,
 'Naturalize': 12.753551483154297,
 'Recollect': 13.35429573059082,
 'Root_Snare': 11.705185890197754,
 'Rupture_Spire': 15.126949310302734,
 'Skymarch_Bloodletter': 16.28285789489746,
 'Skyscanner': 16.772634506225586,
 'Swamp': 9.705110549926758,
 'Viashino_Pyromancer': 15.538342475891113}